In [25]:
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
import tensorflow.python.keras.engine

from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('unprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(3333, random_state=42)) #


ImportError: cannot import name 'Layer' from 'keras.engine' (/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/keras/engine/__init__.py)

In [15]:
df

,SName,Lyric,Genre,Artist,lyric_length
2157,Family Affair,Refrain:. Let's get it crunkupon. We gon' have...,Hip Hop,Mary J. Blige,567
3187,Ransom (ft. Lil' Wayne),"Ransom,. . Yeah,. its Drizzy Baby. you already...",Hip Hop,Drake,967
9164,Close To Me,"T.O.S.. (50 Cent). Unstoppable, incredible, im...",Hip Hop,G-Unit,557
2145,Zone,"uhh, yea. uh uh uh. alright, well alright. . i...",Hip Hop,Drake,400
6442,"Why You Up In Here (feat. Ludacris, Git Fresh ...",Flo-Rida. Gucci!. Bird!. I done bought all thi...,Hip Hop,Flo Rida,524
...,...,...,...,...,...
42982,Paris Bells,"Past the cafÃ© shutters down,. No one stirs in...",Rock,Marianne Faithfull,135
34551,"Lover, Leaver (Taker, Believer)",Oh my holy lord. Sets my soul on fire. Flames ...,Rock,Greta Van Fleet,129
35729,Hunter And The Hunted,Shooting from the hip. About our faith and lov...,Rock,Simple Minds,281
35071,My Champion,May this be your victory song. A song for you ...,Rock,Alter Bridge,250


In [16]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

tf.config.list_physical_devices('GPU') 


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [17]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12861358046774227855
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14461698048
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15647172179220724205
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [18]:

df["label"] = LabelEncoder().fit_transform(df["Genre"])


In [19]:
#!pip install tensorflow_hub
#import tensorflow_hub as hub
#tf.compat.v1.disable_eager_execution()


#


#embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [20]:
#import tensorflow as tf
import pandas as pd
import keras

from sklearn import preprocessing
from keras.utils import np_utils
import numpy as np


y = list(df['label'])
x = list(df['Lyric'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(le, one_hot):
    print(one_hot)
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


x_enc = x
y_enc = encode(le, y)

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.2, random_state=42)

In [22]:
x_train.shape


(7999,)

In [23]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

NameError: name 'Layer' is not defined

In [ ]:
# Function to build model
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(1, activation='sigmoid')(dense)

    model = Model(inputs=[input_text], outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
# Build and fit
model = build_model()
model.fit(x_train, 
          y_train,
          epochs=1,
          batch_size=16)

In [ ]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)

embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text) # elmo vectors are of size 1024. Our first layer (input layer)
x = tf.keras.layers.Dense(1024, activation='relu')(embedding)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)
model = Model(inputs=[input_text], outputs=y)

optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=3, batch_size=16)
    predicts = model.predict(x_test, batch_size=16)
    #model.save_weights('./elmo-model.h5')



#history = model.fit(x_train, y_train, epochs=3, batch_size=16)
#predicts = model.predict(x_test, batch_size=16)

y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [ ]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

from sklearn.metrics import accuracy_score

print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))